For working with the model, we first import some packages and the modelclass:

In [3]:
%run ../stdPackages.ipynb
from pyDbs import *
from py.mBasicPH_storage import *

Exception: File `'../stdPackages.ipynb'` not found.

In [ ]:
read = {'variables': ['Fundamentals', 'LoadVariables', 'GeneratorsVariables','StorageVariables'],
        'maps': ['LoadMaps','MarketMaps','GeneratorsMaps','StorageMaps'],
        'variable2D': ['HourlyVariation']}
db = dbFromWB(os.path.join(d['Data'],'mBasicPH_storage_Data.xlsx'), read)
readSets(db)

In [ ]:
m = mBasicPH_storage.mSimple(db)
m.solve()